In [63]:
from pymongo import MongoClient
from tqdm import tqdm_notebook
from datetime import time, datetime, timedelta

client = MongoClient()
onboard_telemetry_collection = client.sphere_telemetry.onboard

In [67]:
def gps_stamp_to_timedelta(stamp: int) -> timedelta:
	stamp = str(stamp).rjust(6, '0')
	return timedelta(hours=int(stamp[:2]), minutes=int(stamp[2:4]), seconds=int(stamp[4:]))

In [69]:
def write_day_changes():
    f.write(f'\tlocal dt: {prev_local_dt} -> {local_dt}\n')
    f.write(f'\tutc time: {prev_utc_time} -> {utc_time}\n\n')


query = {"GPS_stamp": {"$exists": True}}
total = onboard_telemetry_collection.find(query).count()

prev_local_dt = datetime.fromtimestamp(0)
prev_utc_time = timedelta(hours=23, minutes=59, seconds=59)

utc_tolerance = timedelta(seconds=10)

with open('day_switches.txt', 'w') as f:
    for doc in tqdm_notebook(
        onboard_telemetry_collection.find(query).sort("local_dt"),
        total=total
    ):
        local_dt = doc['local_dt']
        utc_time = gps_stamp_to_timedelta(doc['GPS_stamp'])
        
        if (local_dt.day != prev_local_dt.day) or (utc_time < prev_utc_time - utc_tolerance):
            if local_dt.day != prev_local_dt.day:
                f.write(f'Local date switch!\n')
            if utc_time < prev_utc_time:
                f.write(f'UTC time switch!\n')
            write_day_changes()

        prev_local_dt = local_dt
        prev_utc_time = utc_time


datetime.timedelta(seconds=5963, microseconds=59059)